# Scheduler

There are many situations where you may want to control the number of calcjobs that are running at the same time. For example:
- On the HPC cluster, user may has a limit on the maximum number of submissions that can be running at the same time.
- On the local workstation, user may want to limit the number of calcjobs that are running at the same time to avoid overloading the system.


## Example Usage

Let's walk through an example where we creates four WorkGraphs with five calcjobs each.


In [1]:
from aiida_workgraph import WorkGraph
from aiida import load_profile, orm
from aiida.calculations.arithmetic.add import ArithmeticAddCalculation

load_profile()

# Use the calcjob: ArithmeticAddCalculation
code = orm.load_code("add@localhost")

for i in range(4):
    wg = WorkGraph("test_max_number_jobs")
    # Create N tasks
    for i in range(5):
        temp = wg.add_task(ArithmeticAddCalculation, name=f"add{i}", x=1, y=1, code=code)
        # Set a sleep option for each job (e.g., 10 seconds per job)
        temp.set({"metadata.options.sleep": 10})
    # submit the workgraph to a scheduler called "test_scheduler"
    wg.submit(scheduler="test_scheduler")

WorkGraph process created, PK: 71244
WorkGraph process created, PK: 71245
WorkGraph process created, PK: 71246
WorkGraph process created, PK: 71247


Note, all the WorkGraphs are submitted to a scheduler named `test_scheduler`. The scheduler does not exist yet. Now, let's create the scheduler to limit the number of running calcjobs to 5.

In [2]:
from aiida_workgraph.engine.scheduler.scheduler import Scheduler

scheduler = Scheduler(name="test_scheduler", max_calcjob=5)
scheduler.run()

[INFO] Loaded existing SchedulerNode<69995> for 'test_scheduler'
[INFO] Starting Scheduler 'test_scheduler' main loop. Press Ctrl+C to stop.
[INFO] Summary: waiting_process=0, running_calcjob=0, max_calcjob=5
[INFO] No more processes in waiting queue.
[INFO] Received CONTINUE_TASK for pk=71244
[INFO] Summary: waiting_process=1, running_calcjob=0, max_calcjob=5
[INFO] Best waiting process pk=71244
[INFO] Continuing process pk=71244...
[INFO] No more processes in waiting queue.
[INFO] Received CONTINUE_TASK for pk=71245
[INFO] Summary: waiting_process=1, running_calcjob=0, max_calcjob=5
[INFO] Best waiting process pk=71245
[INFO] Continuing process pk=71245...
[INFO] No more processes in waiting queue.
[INFO] Received CONTINUE_TASK for pk=71246
[INFO] Summary: waiting_process=1, running_calcjob=0, max_calcjob=5
[INFO] Best waiting process pk=71246
[INFO] Continuing process pk=71246...
[INFO] No more processes in waiting queue.
[INFO] Received CONTINUE_TASK for pk=71247
[INFO] Summary: wa

Now, you can check the progress of the WorkGraph using the following command:

```shell
verdi process list
```

This command will display information about the currently running calcjobs, including the fact that there are a maximum of 5 calcjobs running simultaneously.

Additionally, you can monitor the progress visually by visiting `http://127.0.0.1:8000/workgraph`. Here, you can observe the duration of each task's execution over time.

Last but not least, one can stop and restart the scheduler at any time using the same name, all the information will be preserved.